## Lesson 5: Text Generation with Vertex AI

#### Project environment setup

- Load credentials and relevant Python Libraries

In [2]:
# !pip install mplcursors
# !pip install google-cloud-bigquery google-auth
# !pip install python-dotenv
# !pip install ipympl
# !pip install python-dotenv

In [35]:
from utils_5 import authenticate
credentials, PROJECT_ID = authenticate()

In [6]:
REGION = 'us-central1'

### Prompt the model
- We'll import a language model that has been trained to handle a variety of natural language tasks, `text-bison@001`.
- For multi-turn dialogue with a language model, you can use, `chat-bison@001`.

In [7]:
import vertexai
vertexai.init(project=PROJECT_ID,
              location=REGION,
              credentials = credentials)

In [8]:
from vertexai.language_models import TextGenerationModel

In [10]:
import os

# API stuff:
# Do not forget to erase
project = os.env.get["PROJECT_ID"]
REGION = 'us-central1'
# project = project
credentials = os.env.get["credentials"]

In [12]:
# have to add this part back too. Important
# from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform
from google.oauth2 import service_account

# Load the credentials explicitly
credentials = service_account.Credentials.from_service_account_file(
    'google_api.json')

# Initialize Vertex AI with the correct credentials
aiplatform.init(project= project , location="us-central1", credentials=credentials)

In [14]:
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")

## get the api stuff, and get the embedding models (pretrained here)
## now, send a promt and see how the embedding models can generate.

#### Question Answering
- You can ask an open-ended question to the language model.

In [36]:
prompt = "I'm a high school student. \
Recommend me a programming activity to improve my skills."

In [37]:
print(generation_model.predict(prompt=prompt).text)

* **Write a program to solve a problem you're interested in.** This could be anything from a game to a tool to help you with your studies. The important thing is that you're interested in the problem and that you're motivated to solve it.
* **Take a programming course.** There are many online and offline courses available, so you can find one that fits your schedule and learning style.
* **Join a programming community.** There are many online and offline communities where you can connect with other programmers and learn from each other.
* **Read programming books and articles.** There is a


#### Classify and elaborate
- For more predictability of the language model's response, you can also ask the language model to choose among a list of answers and then elaborate on its answer.

In [38]:
prompt = """I'm a high school student. \
Which of these activities do you suggest and why:
a) learn Python
b) learn Javascript
c) learn Fortran
"""
# the maximum token they would generate is 300 :/

In [49]:
print(generation_model.predict(prompt=prompt).text)

As I prepared the picture frame, I reached into my toolkit to fetch my hammer.


#### Extract information and format it as a table

In [40]:
prompt = """ A bright and promising wildlife biologist \
named Jesse Plank (Amara Patel) is determined to make her \
mark on the world.
Jesse moves to Texas for what she believes is her dream job,
only to discover a dark secret that will make \
her question everything.
In the new lab she quickly befriends the outgoing \
lab tech named Maya Jones (Chloe Nguyen),
and the lab director Sam Porter (Fredrik Johansson).
Together the trio work long hours on their research \
in a hope to change the world for good.
Along the way they meet the comical \
Brenna Ode (Eleanor Garcia) who is a marketing lead \
at the research institute,
and marine biologist Siri Teller (Freya Johansson).

Extract the characters, their jobs \
and the actors who played them from the above message as a table
"""

In [41]:
response = generation_model.predict(prompt=prompt)

print(response.text)

| Character | Job | Actor |
|---|---|---|
| Jesse Plank | Wildlife Biologist | Amara Patel |
| Maya Jones | Lab Tech | Chloe Nguyen |
| Sam Porter | Lab Director | Fredrik Johansson |
| Brenna Ode | Marketing Lead | Eleanor Garcia |
| Siri Teller | Marine Biologist | Freya Johansson |


- You can copy-paste the text into a markdown cell to see if it displays a table.

| Character | Job | Actor |
|---|---|---|
| Jesse Plank | Wildlife Biologist | Amara Patel |
| Maya Jones | Lab Tech | Chloe Nguyen |
| Sam Porter | Lab Director | Fredrik Johansson |
| Brenna Ode | Marketing Lead | Eleanor Garcia |
| Siri Teller | Marine Biologist | Freya Johansson |

## Indeed..

| Character | Job | Actor |
|---|---|---|
| Jesse Plank | Wildlife Biologist | Amara Patel |
| Maya Jones | Lab Tech | Chloe Nguyen |
| Sam Porter | Lab Director | Fredrik Johansson |
| Brenna Ode | Marketing Lead | Eleanor Garcia |
| Siri Teller | Marine Biologist | Freya Johansson |

### Adjusting Creativity/Randomness
- You can control the behavior of the language model's decoding strategy by adjusting the temperature, top-k, and top-n parameters.
- For tasks for which you want the model to consistently output the same result for the same input, (such as classification or information extraction), set temperature to zero.
- For tasks where you desire more creativity, such as brainstorming, summarization, choose a higher temperature (up to 1).

In [42]:
temperature = 0.0

In [43]:
prompt = "Complete the sentence: \
As I prepared the picture frame, \
I reached into my toolkit to fetch my:"

In [44]:
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)

In [45]:
print(f"[temperature = {temperature}]")
print(response.text)

[temperature = 0.0]
As I prepared the picture frame, I reached into my toolkit to fetch my hammer.


In [56]:
prompt = "The garden was full of beautiful:"

In [57]:
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)

In [58]:
print(f"[temperature = {temperature}]")
print(response.text)

[temperature = 1.0]
The garden was full of beautiful roses, lilies, and daffodils.


## what's happening here is there are probabilities over all these tokens: flowers, trees, herbs, .... bugs... and they all have a probabiliti, like 0.5, 0.23, 0.22, 0.03.... Which one to choose?
## decoding strategies:
### 1. Greedy decoding; the one with the highest probability. It may generate some interesting or repetitive answer.
### 2. Random sample: Use the probabilities to sample a random token. may end up with some unsual tokens, or some unusual responses. We can control how random it would be though.   One way to control the randomness, it is Temperature.       lower temperartue, it would generate the most likely ones (predictable). If you want something that is less likely, then you'd better choose a higher temperature.

#### softmax: normal softmax   with temperature would become softmax that have a theta (temperature value) at the dinominator under the Z_i (input parameter).

### logit values --> get


#### Temperature of 0: deterministic choice: not creative at all but very reliabe.

### increasing the temperature, it means it would be creative but not very reliable. (completely random)

In [68]:
temperature = 1

In [69]:
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)

In [70]:
print(f"[temperature = {temperature}]")
print(response.text)

[temperature = 1]
The garden was full of beautiful flowers and roses.

The garden is full of beautiful things. We can say: The garden is full of beautiful flowers and roses.


In [61]:
temperature = 0
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)
print(f"[temperature = {temperature}]")
print(response.text)

[temperature = 0]
The garden was full of beautiful flowers.


In [71]:
temperature = 0.5
response = generation_model.predict(
    prompt=prompt,
    temperature=temperature,
)
print(f"[temperature = {temperature}]")
print(response.text)

[temperature = 0.5]
The garden was full of beautiful flowers.

"Full of" means "containing a lot of". The garden was full of beautiful things. We can say "The garden was full of beautiful flowers".


#### Top P
- Top p: sample the minimum set of tokens whose probabilities add up to probability `p` or greater.
- The default value for `top_p` is `0.95`.
- If you want to adjust `top_p` and `top_k` and see different results, remember to set `temperature` to be greater than zero, otherwise the model will always choose the token with the highest probability.


#### top_p: accumulative probabilities.

In [28]:
top_p = 0.2

In [72]:
prompt = "Write an advertisement for jackets \
that involves blue elephants and avocados."

In [73]:
response = generation_model.predict(
    prompt=prompt,
    temperature=0.9,
    top_p=top_p,
)

In [74]:
print(f"[top_p = {top_p}]")
print(response.text)

[top_p = 0.7]
**Introducing the new Blue Elephant Avocado Jacket!**

This jacket is the perfect way to show off your love of both blue elephants and avocados. It's made of a soft, lightweight fabric that's perfect for any season. And the bright colors will make you stand out from the crowd.

The Blue Elephant Avocado Jacket is perfect for any occasion. Wear it to a concert, a party, or even just a casual day out. You're sure to turn heads wherever you go.

So what are you waiting for? Order your Blue Elephant Avocado Jacket today!

**Here are some of the


#### Top k
- The default value for `top_k` is `40`.
- You can set `top_k` to values between `1` and `40`.
- The decoding strategy applies `top_k`, then `top_p`, then `temperature` (in that order).

In [75]:
top_k = 20
top_p = 0.7

In [76]:
response = generation_model.predict(
    prompt=prompt,
    temperature=0.9,
    top_k=top_k,
    top_p=top_p,
)

In [77]:
print(f"[top_p = {top_p}]")
print(response.text)

[top_p = 0.7]
**Introducing the new Blue Elephant Avocado Jacket!**

This jacket is made from soft, durable fabric that will keep you warm and comfortable all winter long. It features a classic button-down design with a front pocket and a hood. The blue color is perfect for any season, and the elephant and avocado print is sure to turn heads.

Whether you're running errands, going to school, or just hanging out, the Blue Elephant Avocado Jacket is the perfect way to stay stylish and warm. Order yours today!

**Here are some of the benefits of owning a Blue Elephant Avocado Jacket:**

*


In [93]:
prompt = "Compare Mac M3 and Studio M2? I am a graduate student in cs, \
which one should I get? only focus on performance. Respond in bullet points."

In [94]:
top_k = 20
top_p = 0.7

response = generation_model.predict(
    prompt=prompt,
    temperature=0.9,
    top_k=top_k,
    top_p=top_p,
)

print(f"[top_p = {top_p}]")
print(response.text)

[top_p = 0.7]
**Mac M3**

* More portable and lightweight
* Better battery life
* Cheaper

**Mac Studio M2**

* More powerful
* More ports
* Better cooling

Overall, the Mac Studio M2 is the better choice for graduate students in CS who need a powerful and performant machine. However, if you are looking for a more portable and affordable option, the Mac M3 is a good choice.
